In [ ]:
# Import Dependencies
import pandas as pd
from census import Census
from config import census_key
import cpi # this will be used to determine the cost of everything in today's dollars. cpi.inflate(amount,year of amount)
#install cpi with pip install cpi on command line
cpi.update()
low_memory=False

In [ ]:
# Import CSVs 
filepath = "Resources/"
kent_zipcodes = pd.read_csv(filepath+"kent_zipcodes.csv",squeeze=True,header=None)
oakland_zipcodes = pd.read_csv(filepath+"oakland_zipcodes.csv",squeeze=True,header=None)
saginaw_zipcodes = pd.read_csv(filepath+"saginaw_zipcodes.csv",squeeze=True,header=None)



In [ ]:
# Create dataframe for each imported CSV and add a column with the corresponding county
kent_zipcodes = pd.DataFrame(kent_zipcodes)
kent_zipcodes["County"] = "Kent"
oakland_zipcodes = pd.DataFrame(oakland_zipcodes)
oakland_zipcodes["County"] = "Oakland"
saginaw_zipcodes = pd.DataFrame(saginaw_zipcodes)
saginaw_zipcodes["County"] = "Saginaw"
# Merge zipcode dataframes and add a column name for Zipcode
query_zipcodes =pd.DataFrame( pd.merge(kent_zipcodes,oakland_zipcodes,how="outer"))
query_zipcodes = pd.DataFrame(pd.merge(query_zipcodes,saginaw_zipcodes,how="outer"))
query_zipcodes.columns = ["Zipcode","County"]
query_zipcodes = query_zipcodes.astype({"Zipcode":object})

In [ ]:
query_zipcodes.info()

In [ ]:
# Create dataframe to add census data to. Needed to put in starter values for it to function properly later. Will remove further down.
census_df = pd.DataFrame({"Year":[0],
                          "Zipcode":["0"],
                          "Total Population":[0],
                          "White Population":[0],
                          "Black Population":[0],
                          "Employed Count":[0],
                          "Unemployed Count":[0],
                          "Poverty Count":[0],
                          "Poverty Count - White":[0],
                          "Poverty Count - Black":[0],
                          "Median Income":[0],
                          "Median Rent":[0],
                          "Median Home Value":[0]})

# Get data from API for Census
census_df = census_df.astype({"Year":int,"Zipcode":object})

# For loop to iterate through the desired 10-year period
for year in [2011,2012,2013,2014,2015,2016,2017,2018,2019,2020]: 

    c = Census(census_key,year) 
    census_data = c.acs5.get(("NAME", "B19013_001E", "B23025_004E", "B23025_005E", "B17001_002E", 
            "B17001A_002E","B17001B_002E", "B01003_001E",
            "B02001_002E", "B02001_003E","B25058_001E", "B25077_001E"
             ), {"for": "zip code tabulation area:*"})
    census_raw_df = pd.DataFrame(census_data)
    
# Add the year to the year column so each iteration can be distinguished
    year_count = len(census_data)
    year_series = pd.Series([year for x in range(year_count)])    
    census_raw_df["Year"] = year_series
    
# Rename Columns
    census_raw_df = census_raw_df.rename(columns = {
            "B19013_001E" : "Median Income", 
            "B23025_004E" : "Employed Count", 
            "B23025_005E" : "Unemployed Count", 
            "B17001_002E" : "Poverty Count", 
            "B17001A_002E" : "Poverty Count - White" , 
            "B17001B_002E" : "Poverty Count - Black" , 
            "B01003_001E" : "Total Population", 
            "B02001_002E" : "White Population", 
            "B02001_003E" : "Black Population", 
            "B25058_001E" : "Median Rent", 
            "B25077_001E" : "Median Home Value", 
            "NAME" : "Name",
            "zip code tabulation area": "Zipcode"})
    
# Recast all variables in the output dataframe to the correct type 
    census_raw_df = census_raw_df.astype({"Year":int,"Zipcode":object,"Median Income":float,"Employed Count":float,"Unemployed Count":float,"Poverty Count":float,
             "Poverty Count - White":float, "Poverty Count - Black":float, "Total Population":float,
             "White Population":float,"Black Population":float, "Median Rent":float,"Median Home Value":float})
    
# Recast the Zipcode in census_df to object. Was running into issues with the zipcode repeatedly trying to cast as a float.
    census_df = census_df.astype({"Zipcode":object})
    
# Merge the new raw df into the finished df. Use outer to maintain all values
    census_df = pd.merge(census_df,census_raw_df,how="outer")

In [ ]:
for i in query_zipcodes.index:
    query_zipcodes.at[i,"Zipcode"] = str(query_zipcodes.loc[i,"Zipcode"])
query_zipcodes

In [ ]:
# Merge Dataframes so only those zipcodes we want are there. Also adds County 
census_df = pd.merge(census_df,query_zipcodes,how="inner", on="Zipcode")

In [ ]:
# Remove the name column, which is useless here, and the state column, which only functioned for some years
census_df = census_df.drop(columns = ["Name","state"])

#Remove the sample data added to initialize the dataframe
census_df = census_df.loc[census_df["Year"] != 0]

In [ ]:
census_df.sort_values("Median Income")

In [ ]:
census_df = census_df.loc[(census_df["Zipcode"]!= "48724") & (census_df["Zipcode"]!= "48370") & (census_df["Zipcode"]!= "48380")]

In [ ]:
census_df.sort_values("Median Rent").head(20)

In [ ]:
# Adjust financial columns to account for inflation. All values go to current-day inflation (2023)
for i in census_df.index:
    census_df.at[i,"Adjusted Median Income"] = cpi.inflate((census_df.at[i,"Median Income"]),(census_df.at[i,"Year"]))
    census_df.at[i,"Adjusted Median Rent"] = cpi.inflate((census_df.at[i,"Median Rent"]),(census_df.at[i,"Year"]))
    census_df.at[i,"Adjusted Median Home Value"] = cpi.inflate((census_df.at[i,"Median Home Value"]),(census_df.at[i,"Year"]))


In [ ]:
census_df.to_csv("output_data/census.csv",index_label="datapoint")